<a href="https://colab.research.google.com/github/talhaanwarch/OffenseEval2020/blob/master/Danish/FastText_biGRU_crossval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Danish Language


In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!pip install focal-loss
!pip install keras-radam

  Created wheel for keras-radam: filename=keras_radam-0.15.0-cp36-none-any.whl size=14685 sha256=58edb1b996bc9fc338564b1979ab455dd4bc026800cffb555e037d1f5a5ff6eb
  Stored in directory: /root/.cache/pip/wheels/79/a0/c0/670b0a118e8f078539fafec7bd02eba0af921f745660c7f83f
Successfully built keras-radam


In [6]:
cd /content/drive/My Drive/dataset/OffenseEval2020/data/Danish/

/content/drive/My Drive/dataset/OffenseEval2020/data/Danish


In [7]:
ls

cc.da.300.vec  cc.da.300.vec.gz  offenseval-da-training-v1.tsv


In [0]:
import pandas as pd
train=pd.read_csv( 'offenseval-da-training-v1.tsv',sep="\t")
train.drop(train.tail(n=1).index,inplace=True)
train=train.sample(frac=1)

In [19]:
train.head()

,id,tweet,subtask_a
1183,2175,Puha var lige ved at falde om!,NOT
750,1613,Hvad skal den titel betyde?,NOT
2437,2588,Jävla dansksvin,OFF
176,3376,Se dét er en god migmig!,NOT
2063,1648,Jeg troede [den her](URL,NOT


In [20]:
train.tail()

,id,tweet,subtask_a
1296,1762,"|rejser sig omgående, triller en patriotisk tå...",NOT
1119,139,mon ikke at det samme kunne ske hvis manden in...,NOT
1104,1309,@USER ro og orden i Afrika. tror du selv på de...,OFF
2176,3279,"Godt gået, ven!",NOT
2595,2060,For fanden da... Lad os hjælpe ham med at stjæ...,NOT


In [0]:
labels=train['subtask_a']
train['tweet'] = train['tweet'].str.replace('\d+', '')
train=train['tweet']
labels=pd.factorize(labels)[0]

In [22]:
import collections
collections.Counter(labels)

Counter({0: 2576, 1: 384})

In [0]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
max_words = 10000 #frequency of words to be kept
max_len = 200

tokenize = Tokenizer(num_words=max_words)
tokenize.fit_on_texts(train)
sequences = tokenize.texts_to_sequences(train)
word_index = tokenize.word_index
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [24]:
len(sequences_matrix),len(labels)

(2960, 2960)

In [0]:
from keras.models import Sequential
from keras.layers import Embedding,CuDNNGRU,GRU,Dense,Dropout,Bidirectional,GlobalMaxPool1D,GlobalAveragePooling1D, SpatialDropout1D
from keras.optimizers import RMSprop
from keras.initializers import Constant
from sklearn.utils import class_weight
from sklearn.metrics import classification_report,f1_score
import keras
import numpy as np
from sklearn.model_selection import StratifiedKFold
from focal_loss import BinaryFocalLoss
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

In [26]:
num_words = min(max_words, len(word_index)) + 1
print(num_words)

9813


In [0]:
# !wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.el.300.vec.gz

In [0]:
# import gzip
# import shutil
# with gzip.open('cc.el.300.vec.gz', 'rb') as f_in:
#     with open('cc.el.300.vec', 'wb') as f_out:
#         shutil.copyfileobj(f_in, f_out)

In [0]:
embed_size=300
def get_coefs(word,*arr):
    return word, np.asarray(arr, dtype='float32')

def build_matrix(embedding_path, word_index):
    embedding_index = dict(get_coefs(*o.strip().split(" ")) for o in open(embedding_path))

    nb_words = min(max_words, len(word_index))
    embedding_matrix = np.zeros((nb_words + 1, embed_size))
    for word, i in word_index.items():
        if i >= max_words:
            continue
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

In [0]:
embeddings=build_matrix('cc.da.300.vec', word_index)


In [31]:
embeddings.shape

(9813, 300)

In [0]:
def model_func():
  model = Sequential()
  model.add(Embedding(num_words,embed_size,embeddings_initializer=Constant(embeddings),input_length=max_len,trainable=False))
  model.add(Bidirectional(CuDNNGRU(100, return_sequences = True)))
  #model.add(Bidirectional(CuDNNGRU(32, return_sequences = True)))
  model.add(GlobalMaxPool1D())
  model.add(Dense(100, activation="relu"))
  model.add(Dense(120, activation="relu"))
  model.add(Dense(10, activation="relu"))

  model.add(Dropout(0.05))
  model.add(Dense(1, activation="sigmoid"))
  model.compile(loss=BinaryFocalLoss(gamma=2), optimizer='adam', metrics=['accuracy'])
  return model

In [0]:
import numpy as np
class_weights = class_weight.compute_class_weight('balanced',np.unique(labels),labels)
class_weights=dict(enumerate(class_weights))

In [0]:
from keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=2, min_lr=0.000001, verbose=1)

In [40]:
kfold = StratifiedKFold(n_splits=10, shuffle=False, random_state=2020)
cvscores = []

for train, test in kfold.split(sequences_matrix, labels):
  model=model_func()
  
  model.fit(sequences_matrix[train], labels[train],batch_size=256,epochs=10,verbose=2,
            validation_data=(sequences_matrix[test],labels[test]),callbacks=[reduce_lr])
  y_pred = model.predict(sequences_matrix[test], batch_size=128, verbose=1)
  y_pred_bool = np.argmax(y_pred, axis=1)
  y_pred = (y_pred > 0.5)
  f1=f1_score(labels[test], y_pred, average='macro')
  cvscores.append(f1)
  keras.backend.clear_session()

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


Train on 2664 samples, validate on 296 samples
Epoch 1/10
 - 1s - loss: 0.1276 - acc: 0.8686 - val_loss: 0.1009 - val_acc: 0.8716
Epoch 2/10
 - 1s - loss: 0.1113 - acc: 0.8701 - val_loss: 0.0991 - val_acc: 0.8716
Epoch 3/10
 - 1s - loss: 0.1047 - acc: 0.8701 - val_loss: 0.0977 - val_acc: 0.8716
Epoch 4/10
 - 1s - loss: 0.0981 - acc: 0.8701 - val_loss: 0.0942 - val_acc: 0.8716
Epoch 5/10
 - 1s - loss: 0.0887 - acc: 0.8701 - val_loss: 0.0879 - val_acc: 0.8716
Epoch 6/10
 - 1s - loss: 0.0795 - acc: 0.8701 - val_loss: 0.0814 - val_acc: 0.8716
Epoch 7/10
 - 1s - loss: 0.0722 - acc: 0.8701 - val_loss: 0.0786 - val_acc: 0.8716
Epoch 8/10
 - 1s - loss: 0.0657 - acc: 0.8784 - val_loss: 0.0777 - val_acc: 0.8682
Epoch 9/10
 - 1s - loss: 0.0625 - acc: 0.9084 - val_loss: 0.0862 - val_acc: 0.8986
Epoch 10/10
 - 1s - loss: 0.0552 - acc: 0.9129 - val_loss: 0.0835 - val_acc: 0.8885

Epoch 00010: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
296/296 [==============================]

In [41]:
np.array(cvscores).mean()

0.7078057510821537